## Section 1: Load mask files from directory

In [4]:
import os
from os import listdir
from skimage.io import imread

# get the files in the path/directory where the images are stored
folder_dir = "C:/Users/jonatmt/OneDrive - Universitetet i Oslo/Desktop/Amalie masks"
files = os.listdir(folder_dir)

# Make separate lists of the filepaths for images
ER_Files = []
Vinc_Files = []
ER_Names = []
Vinc_Names = []

for File in files:
    if File.endswith(".tif"):
        if "threshold-ER" in File:
            ER_Files.append(os.path.join(folder_dir, File))
            ER_Names.append(File)
        elif "threshold-vinc" in File:
            Vinc_Files.append(os.path.join(folder_dir, File))
            Vinc_Names.append(File)

# Function to extract base name without the threshold type
def get_base_name(file_name):
    # Remove the prefix (e.g., C2- or C3-)
    if file_name.startswith(("C2-", "C3-")):
        file_name = file_name.split("-", 1)[1]
    
    # Remove the "-threshold" part
    return file_name.rsplit("-threshold", 1)[0]

# Creating a dictionary to store pairs
paired_masks = {}

for er_name, er_path in zip(ER_Names, ER_Files):
    base_name = get_base_name(er_name)
    paired_masks[base_name] = {'ER': er_path, 'Vinc': None}

for vinc_name, vinc_path in zip(Vinc_Names, Vinc_Files):
    base_name = get_base_name(vinc_name)
    if base_name in paired_masks:
        paired_masks[base_name]['Vinc'] = vinc_path
    else:
        paired_masks[base_name] = {'ER': None, 'Vinc': vinc_path}

# Display all pairs and check if any are missing
for base_name, paths in paired_masks.items():
    print(f"{base_name}: ER: {paths['ER'] is not None}, Vinc: {paths['Vinc'] is not None}")


250123_siRab18_53_control_ER-GFP_Vinc-RFP_cell04: ER: True, Vinc: True
250123_siRab18_53_L24Q-BFP_ER-GFP_Vinc-RFP_cell10: ER: True, Vinc: True
250123_siRab18_53_Scr-control_ER-GFP_Vinc-RFP_cell4: ER: True, Vinc: True
250123_siRab18_53_WT-BFP_ER-GFP_Vinc-RFP_cell08b: ER: True, Vinc: True


## Section 2: Quantify the FA properties and ER overlap, save data as csv file 

In [ ]:
import pandas as pd
import os
import numpy as np
import napari_segment_blobs_and_things_with_membranes as nsbatwm
from skimage import measure
from skimage.io import imread

def get_region_property_values(region, property_names):
    property_values = {}
    for prop_name in property_names:
        prop_value = getattr(region, prop_name, None)
        if ' ' not in str(prop_value):
            property_values[prop_name] = prop_value
    return property_values

# Analysis loop for each pair in paired_masks
for base_name, paths in paired_masks.items():
    ER_file = paths['ER']
    Vinc_file = paths['Vinc']

    if not ER_file or not Vinc_file:
        print(f"Skipping {base_name}: Missing ER or Vinc file.")
        continue  # Skip if one of the files is missing

    # Load the images
    ERThresh = imread(ER_file)
    VincThresh_pro = imread(Vinc_file)
    
    ## Analysis of FA excluding the ERsheets region
    VincAna = nsbatwm.connected_component_labeling(VincThresh_pro, False)
    props = measure.regionprops(VincAna, intensity_image=VincThresh_pro)
    property_names = [attr for attr in dir(props[0]) if not attr.startswith('_')]
    region_data = [get_region_property_values(region, property_names) for region in props]

    intersection_percentages = []
    intersection_areas = []
    num_intersection_areas = []

    for region in props:
        region_label = region.label
        region_mask = VincAna == region_label
        intersection = np.logical_and(region_mask, ERThresh)
        intersection_area = np.sum(intersection)
        region_area = region.area

        intersection_percentage = (intersection_area / region_area) * 100 if region_area > 0 else 0.0
        intersection_components, num_intersections = measure.label(intersection, return_num=True)

        intersection_percentages.append(intersection_percentage)
        intersection_areas.append(intersection_area)
        num_intersection_areas.append(num_intersections)

    for i, region_dict in enumerate(region_data):
        region_dict['ERThresh_intersection_percentage'] = intersection_percentages[i]
        region_dict['ERThresh_intersection_area'] = intersection_areas[i]
        region_dict['ERThresh_num_intersection_areas'] = num_intersection_areas[i]

    df = pd.DataFrame(region_data)
    file_name_csv = f"{base_name}_NoERsheets.csv"
    file_path = os.path.join(folder_dir, file_name_csv)
    df.to_csv(file_path, index=False)


    ## Print a progress update
    print(f"Analysis complete for {base_name}. Results saved.")



Analysis complete for 250123_siRab18_53_control_ER-GFP_Vinc-RFP_cell04. Results saved.
Analysis complete for 250123_siRab18_53_L24Q-BFP_ER-GFP_Vinc-RFP_cell10. Results saved.
Analysis complete for 250123_siRab18_53_Scr-control_ER-GFP_Vinc-RFP_cell4. Results saved.
Analysis complete for 250123_siRab18_53_WT-BFP_ER-GFP_Vinc-RFP_cell08b. Results saved.
